In [1]:
import pandas as pd
import numpy as np
import nltk
import gensim 
import string
import pickle

pd.set_option('display.max_colwidth', 150)


fr = open("garbage.txt", 'rb')
garbage = pickle.load(fr)
garbage = [t[0] for t in garbage]
fr.close()

In [2]:
def getTokens(all_command):
    handle_all_command = []
    lines = all_command.tolist()
    stopwords = set(nltk.corpus.stopwords.words("english"))
    table = str.maketrans("","",string.punctuation)
    for line in lines:
        line = " ".join([word for word in str(line).split(" ") if '@' not in word and 'http' not in word and "RT" not in word])
        tokens = nltk.word_tokenize(line)
        tokens = [w.lower() for w in tokens]
        tokens = [w.translate(table) for w in tokens]
        tokens = [w for w in tokens if w.isalpha()]
        tokens = [w for w in tokens if w not in stopwords and w not in garbage]
        handle_all_command.append(tokens)
    return handle_all_command

In [3]:
needExplore = pd.read_csv("newPredict.csv")
needExplore.columns

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['Date', 'Time', 'Training/Test', 'preds', 'pred_probs', 'external_url',
       'numbers', 'external_url_title', 'New_Title', 'ab_clean', 'Title',
       'Abstract', 'Link', 'tweet_id', 'zero_length', 'hashtags',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_text', 'medias', 'mentions', 'tweet_found',
       'user_name', 'nouns', 'nnp', 'word_recall', 'word_alert',
       'word_allergy', 'word_outbreak', 'hashtag_RECALL', 'hashtag_Recall',
       'hashtag_SmartNews', 'mention_googlenews', 'num_words',
       'user_name_words', 'user_name_word_News', 'has_media', 'question_mark',
       'neg_noun_I', 'neg_noun_you', 'neg_noun_i', 'neg_noun_me',
       'neg_noun_we', 'neg_noun_they', 'neg_noun_It', 'neg_noun_You'],
      dtype='object')

In [4]:
needExplore.loc[needExplore["external_url_title"].notnull(), 'New_Title'] = needExplore[needExplore["external_url_title"].notnull()]["external_url_title"]
needExplore.loc[needExplore["in_reply_to_status_text"].notnull(), 'New_Title'] = needExplore[needExplore["in_reply_to_status_text"].notnull()]["in_reply_to_status_text"]                                                                                                                                                                                                                                       

# 因为爬取的title有大量的 XXX on Twitter，需要处理下

In [5]:
temp = []
for text in needExplore.New_Title.astype(str):
    index = text.find(":")
    if index != -1:
        temp.append(text[index + 1:])
    else:
        temp.append(text)
temp

[' Allergy alert (Vic) - Peter Bouchier Thai Chicken Stir Fry (Use by 9/03/19) is being recalled due to an undeclared allergenâ€¦',
 'Food allergy recall of FUISA Chicken Fried Rice meal | Miami Herald',
 'Food allergy recall of FUISA Chicken Fried Rice meal | Miami Herald',
 ' "#RECALL ALERT: Butterball recalls more than 78,000 pounds of ground turkey due to salmonella concerns https://t.co/rx9HkzPXXpâ€¦ https://t.co/K781hy5oF0"',
 ' "#RECALL: Pillsbury flour recall announced due to salmonella concerns https://t.co/iPf70gd3Z2â€¦ "',
 ' "#RECALL More than 12,000 cases of Pillsbury flour recalled over salmonella concerns\n\nMore: https://t.co/SPLXWwn4wLâ€¦ "',
 ' "Recall alert: Publix, Winn-Dixie announce Pillsbury flour recall https://t.co/1bk6RNtTmuâ€¦ "',
 ' "RECALL ALERT: Pillsbury flour recall announced due to salmonella concerns https://t.co/wNiteU331iâ€¦ "',
 ' "RECALL ALERT for Pillsbury flour due to salmonella concerns   MORE INFO: https://t.co/2KTdWuymXKâ€¦ "',
 ' "RECALL ALER

In [6]:
needExplore['New_Title'] = temp
tokens = getTokens(needExplore['New_Title'].astype(str))
needExplore['new_tokens'] = tokens
needExplore['len'] = [len(token) for token in tokens]

# 总真的个数2143， 去掉小于3后的个数2084

In [7]:
currTrueData = needExplore[needExplore['preds'] == True]
currTrueData = currTrueData[currTrueData['len'] >= 3]
sent = currTrueData['new_tokens']

In [8]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary


dic = Dictionary(sent)
corpus = [dic.doc2bow(text) for text in sent]

%time lda = LdaModel(corpus=corpus, id2word=dic, num_topics=16, passes=100, random_state=42) 


CPU times: user 48.9 s, sys: 110 ms, total: 49 s
Wall time: 49.1 s


In [9]:
top_3_topic_score = []
for x in lda[corpus]:
    top_3_topic_score.append(sorted(x, key=lambda y: y[1], reverse=True)[0: 3])

In [10]:
currTrueData['top_3_topic_score'] = top_3_topic_score
currTrueData['predID'] = [tup[0][0] for tup in top_3_topic_score]

In [17]:
toCsv = currTrueData[["preds", "predID", "top_3_topic_score", "New_Title", "external_url_title", "new_tokens", "ab_clean"]]
toCsv = toCsv.sort_values("predID")


In [18]:
toCsv.to_csv("16_topics.csv", index=False)

In [13]:
lda.print_topics(num_topics=16, num_words=10)

[(0,
  '0.050*"potential" + 0.029*"two" + 0.024*"food" + 0.018*"boston" + 0.018*"allpurpose" + 0.016*"could" + 0.016*"grocery" + 0.014*"salmonella" + 0.013*"dry" + 0.013*"gets"'),
 (1,
  '0.096*"due" + 0.095*"risk" + 0.093*"recall" + 0.074*"foodsafety" + 0.036*"announced" + 0.033*"select" + 0.031*"packages" + 0.026*"day" + 0.023*"every" + 0.022*"ðÿ"'),
 (2,
  '0.171*"missed" + 0.140*"recall" + 0.084*"free" + 0.036*"chips" + 0.028*"stupid" + 0.027*"lose" + 0.027*"heard" + 0.027*"planes" + 0.027*"get" + 0.014*"pa"'),
 (3,
  '0.052*"product" + 0.037*"recalls" + 0.033*"gluten" + 0.029*"food" + 0.028*"products" + 0.024*"people" + 0.023*"important" + 0.023*"foodsafety" + 0.022*"wheat" + 0.021*"brands"'),
 (4,
  '0.074*"recall" + 0.048*"flour" + 0.048*"dates" + 0.047*"foods" + 0.046*"look" + 0.044*"learned" + 0.044*"ravioli" + 0.043*"pillsbury" + 0.042*"journey" + 0.041*"child"'),
 (5,
  '0.115*"like" + 0.106*"food" + 0.096*"recalls" + 0.074*"unsafe" + 0.066*"bakery" + 0.065*"montreal" + 0.02